In [1]:
!pip -q install google-cloud-aiplatform google-api-python-client
!pip -q install langchain chromadb==0.3.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.5/964.5 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.

In [1]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
#from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
#from langchain.document_loaders import DirectoryLoader

Install pypdf library to load pdf files

In [45]:
pip install pypdf

In [4]:
loader = PyPDFLoader("/content/CS_982_Assignment.pdf")
pages = loader.load_and_split()

the load and split method will split the document based on pages, the uploaded document has 20 pages

In [7]:
len(pages)

20

Next I have further split the document using the Recursive splitter function

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(pages)

In [14]:
len(texts)

36

In [8]:
pip install --upgrade --quiet  langchain-google-genai

Obtain Google API key from https://makersuite.google.com/app/apikey to use the latest Generative AI model Gemini pro

In [10]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass ("Provide your Google API key here")


Provide your Google API key here··········


I have used Generative AI Embeddings to transform sentences to embeddings

In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
#vector = embeddings.embed_query("hello, world!")

There are separate methods to embed documents and queries

query_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", task_type="retrieval_query"
)
doc_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001", task_type="retrieval_document"
)





Next I have created a vector store on the disk, other options are creating vector store on memory FASS and cloud- Pinecone

In [15]:
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

Testing the vector store with some semantic search queries

In [18]:
query = "Are airlines industry profitable"
docs = vectordb.similarity_search(query)

In [19]:
for doc in docs:
    print (f"{doc.page_content}\n")

CS982: Big Data Technologies  
Report on Airline Passenger Satisfaction survey

3 
 Chapter 1  :Introduction  & Problems addressed : 
 
The airline  industry has seen tremendous growth over the past few decades , and before the 
pandemic , the industry served 4.5 billion  passengers on scheduled s ervices (The World of Air 
Transport in 2019 , 2020) . This upward  trend saw major changes  due to  the recent Covid19 
outbreak , during which the airline industry suffer ed massive losses . As per some estimates , the 
industry suffered a loss of $372 billion in 2020 alone (Effects of Novel Coronavirus (COVID -19) on 
Civil Aviation: Economic Impact Analysis Economic Development -Air Transport Bureau , 2022) .  
However, it is interesting to note that even before the pandemic a number of Airline companies 
including Kingfisher airlines, Transaero  airlines  and many more had either filed for bankruptcy or 
shut down  their operations after incurring huge losses in their business  (Famous A

Below I have setup a retriever object which can be used within RetrieveQA method

In [29]:
retriever = vectordb.as_retriever(search_kwargs={"k":4})

In [41]:
docs2 = retriever.get_relevant_documents("what are the top services in the flight")

In [42]:
for doc in docs2:
    print (f"{doc.page_content}\n")

10 
  Continuing our analysis, we will next try to understand  the best and the worst of airlines’  
services. One way to know the best and worst service  is by examining which services received 
the highest 5-star and 1 -star ratings respectively. This is better illustrated by Figure 3 -6 below:  
 
Figure 3-6 Best and Worst services  
From the above figures we can clearly identify that the top 3 best services are In -flight service, 
followed by baggage handling and Seat Comfort. The airlines have to ensure that the quality of 
these services remains the same over time. From the same figure, we can also identify the top 3 
worst services as In-Flight Wi -Fi Service followed by Gate location  and Ease of Online Booking.  
The airline clearly needs to address the  quality of  In-flight Wi -Fi se rvice by switching to a better 
Internet Service Provider. They also need to spend time on improving the User Experience on

5 
 • Flight Distance : Flight distance in miles  
• Departure Delay

The LLM used to extract relevant info from the documents returned from vector DB here is Google's gemini pro

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [30]:
llm = ChatGoogleGenerativeAI(model='gemini-pro',
                    max_output_tokens=256,
                    temperature=0.1,
                    top_p=0.8,
                    top_k=40,convert_system_message_to_human=True)

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [43]:
query = "what are the top services in the flight"
llm_response = qa_chain(query)
llm_response["result"]

'The top 3 best services are In-flight service, followed by baggage handling and Seat Comfort.'

In [47]:
llm_response['source_documents']

[Document(page_content='10 \n  Continuing our analysis, we will next try to understand  the best and the worst of airlines’  \nservices. One way to know the best and worst service  is by examining which services received \nthe highest 5-star and 1 -star ratings respectively. This is better illustrated by Figure 3 -6 below:  \n \nFigure 3-6 Best and Worst services  \nFrom the above figures we can clearly identify that the top 3 best services are In -flight service, \nfollowed by baggage handling and Seat Comfort. The airlines have to ensure that the quality of \nthese services remains the same over time. From the same figure, we can also identify the top 3 \nworst services as In-Flight Wi -Fi Service followed by Gate location  and Ease of Online Booking.  \nThe airline clearly needs to address the  quality of  In-flight Wi -Fi se rvice by switching to a better \nInternet Service Provider. They also need to spend time on improving the User Experience on', metadata={'source': '/content/CS